In [ ]:
%matplotlib

In [ ]:
%matplotlib inline

In [ ]:
pd.set_option('display.max_rows',    100)      # DataFrame 데이터 확인 시 최대 표시 행 수
pd.set_option('display.max_columns', 100)      # DataFrame 데이터 확인 시 최대 표시 열 수

## 라이브러리 import

In [1]:
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf

tf.__version__                         # tensorflow 버전 확인

'2.5.0'

Tensorflow v1 기능만 사용 (v2 기능 강제 차단)

In [3]:
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior() #v2 기능 강제 차단
tf.set_random_seed(777)  # 결과 재현성을 위하여 난수 시드 지정

Instructions for updating:
non-resource variables are not supported in the long term


## 데이터 불러오기

In [4]:
Fold = 5

# k-fold 학습/검증 데이터
for i in range(Fold):
    
    path1 = './K_FoldData/Training_Fold%d'%(i+1)
    path2 = './K_FoldData/Validation_Fold%d'%(i+1)
    c1 = 'Training_Fold%d   = np.array(pd.read_csv(path1, sep=",", header=None))'%(i+1)
    c2 = 'Validation_Fold%d = np.array(pd.read_csv(path2, sep=",", header=None))'%(i+1)
    exec(c1)
    exec(c2)

# K-fold 학습/검증 레이블
TrainingFold_Label   = np.array(pd.read_csv('./K_FoldData/TrainingFold_Label_forANN'  , sep=",", header=None))
ValidationFold_Label = np.array(pd.read_csv('./K_FoldData/ValidationFold_Label_forANN', sep=",", header=None))
    
    
# 전체 학습용 데이터
Training_All       = np.array(pd.read_csv('./K_FoldData/Training_All', sep = ",", header = None))
Training_All_Label = np.array(pd.read_csv('./K_FoldData/Training_All_Label_forANN', sep = ",", header = None))

print(Training_Fold1.shape)
print(Validation_Fold1.shape)
print(TrainingFold_Label.shape)
print(ValidationFold_Label.shape)
print(Training_All.shape)

(256, 30)
(64, 30)
(256, 2)
(64, 2)
(320, 30)


## ANN(Artificial Neural Network) hyperparameter 설정

In [5]:
learningRate  = 0.0001
noOfNeuron    = 16 #레이어당 뉴런수
iteration     = 5000
#act_func = relu, sigmoid 이런 파라미터도 있다

## ANN 구조(Architecture) 설계

In [6]:
# input place holders
X = tf.placeholder(tf.float32, [None, Training_Fold1.shape[1]]) # 데이터 타입 정의, 몇개의 데이터가 들어올건지 여기서는 30개
Y = tf.placeholder(tf.float32, [None, 2]) # one hot encoding 으로 지정해서 2차원으로 만듬

# dropout (keep_prob) rate 0.7 on training, but should be 1 for testing 모델의 과적합 방지를 위해 한번 이터레이션이 돌때 비활성화되는 뉴런을 줄인다
# keep_prob = tf.placeholder(tf.float32)                                                       이런 기법이 있다 정도만

# weights & bias for nn layers

# Input Layer
W1 = tf.Variable(tf.random_normal([Training_Fold1.shape[1], noOfNeuron])) #30개 16개
b1 = tf.Variable(tf.random_normal([noOfNeuron])) #뉴런의 바이어스만금만
L1 = tf.matmul(X, W1) + b1 #인풋값과 생성된 weight값들을 곱해주고 바이어스를 더해주면 인풋레이어의 아웃풋이 나옴

# Hidden Layer 1
W2 = tf.Variable(tf.random_normal([noOfNeuron, noOfNeuron])) #첫번째 레이어 뉴런의 개수가 인풋 두번째 레이어의 뉴런의 개수
b2 = tf.Variable(tf.random_normal([noOfNeuron])) #뉴런 하나마다 가지고 있는 가중치
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2) #인풋레이어의 아웃풋에 가중치 곱해주고 바이어스 더해줌 그리고 액티베이션 펑션 적용 

# Hidden Layer 2
W3 = tf.Variable(tf.random_normal([noOfNeuron, noOfNeuron]))
b3 = tf.Variable(tf.random_normal([noOfNeuron]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3) #은닉층 2개 통과한 output

# Output Layer
W4 = tf.Variable(tf.random_normal([noOfNeuron, 2]))
b4 = tf.Variable(tf.random_normal([2]))
hypothesis = tf.matmul(L3, W4) + b4 #최종 output

# define cost/loss & optimizer
cost      = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y)) #비용함수 정의 hypothesis 평균이 최소화 되는거
optimizer = tf.train.AdamOptimizer(learning_rate=learningRate).minimize(cost) #아담 옵티마이저를 사용, 학습률 지정, 코스트값 집어 넣는다

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy           = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## NN(Neural Network) 학습 및 평가 방법

In [7]:
Data      = Training_Fold1 
Data_Val  = Validation_Fold1
Label     = TrainingFold_Label
Label_Val = ValidationFold_Label
#학습, 검즘, 레이블 불러옴

sess = tf.Session()
sess.run(tf.global_variables_initializer()) #전역변수 초기화

for step in range(1, iteration+1):
    c, o = sess.run([cost, optimizer], feed_dict = {X : Data, Y: Label}) #run함수를 통해
    if step % 500 == 0:
        print('Cost at {}th iteration: {}'.format(step, c))

Validation_Acc_Fold1= sess.run(accuracy, feed_dict={X : Data_Val, Y: Label_Val})
print('Fold 1 Validation Acc: {:.2f}%'.format(Validation_Acc_Fold1*100))

sess.close() #반복됨에 따라 cost낮아짐 정확도는 95.31%

Cost at 500th iteration: 1.3772462606430054
Cost at 1000th iteration: 1.2566884756088257
Cost at 1500th iteration: 1.0911749601364136
Cost at 2000th iteration: 0.9011598825454712
Cost at 2500th iteration: 0.6582923531532288
Cost at 3000th iteration: 0.43442022800445557
Cost at 3500th iteration: 0.29074928164482117
Cost at 4000th iteration: 0.1346479207277298
Cost at 4500th iteration: 0.0911049172282219
Cost at 5000th iteration: 0.07932459563016891
Fold 1 Validation Acc: 95.31%


## NN(Neural Network) 학습 및 검증

In [8]:
Label     = TrainingFold_Label
Label_Val = ValidationFold_Label
Accuracy_sum = 0

print('[Result of K-fold Cross Validation] \n')

for i in range(Fold):
    
    s1= 'Data     = Training_Fold%d'  %(i+1)
    s2= 'Data_Val = Validation_Fold%d'%(i+1)
    exec(s1)
    exec(s2)

# train my model
# initialize
    sess = tf.Session() #폴드돌때 마다 초기화
    sess.run(tf.global_variables_initializer())

    for step in range(1, iteration+1):
    
        c, o = sess.run([cost, optimizer], feed_dict = {X : Data, Y: Label})
        
    Accuracy = sess.run(accuracy, feed_dict={X : Data_Val, Y: Label_Val})
    Accuracy_sum = Accuracy_sum + Accuracy
    
    print('Fold {}: {:.2f}%'.format((i+1), Accuracy*100))
    sess.close()
    
print('* Average accuracy : {:.2f}%'.format((Accuracy_sum/Fold)*100))

[Result of K-fold Cross Validation] 

Fold 1: 95.31%
Fold 2: 95.31%
Fold 3: 100.00%
Fold 4: 96.88%
Fold 5: 100.00%
* Average accuracy : 97.50%


## 전체 학습 데이터로 NN 학습 및 모델 저장

In [9]:
x_data = Training_All
y_data = Training_All_Label

In [10]:
# train my model
# initialize
sess = tf.Session() #초기화 해줌
sess.run(tf.global_variables_initializer())

for step in range(1, iteration+1):
    
    c, o = sess.run([cost, optimizer], feed_dict = {X : x_data, Y: y_data}) #코스트랑 옴티마이저 동일, x,y데이터로 사용하는거 동일
    Acc = sess.run(accuracy, feed_dict={X : x_data, Y: y_data})
    
    if step % 1000 == 0:
            
        print('iteration {}'.format(step))    
        print('   Validation Acc: {:.2f}%'.format(Acc*100))
        

iteration 1000
   Validation Acc: 94.38%
iteration 2000
   Validation Acc: 95.00%
iteration 3000
   Validation Acc: 95.94%
iteration 4000
   Validation Acc: 97.19%
iteration 5000
   Validation Acc: 98.44%


In [17]:
# 모델 저장
saver = tf.train.Saver()
saver.save(sess, './MLmodels/ANN_model_tf1')

'./MLmodels/ANN_model_tf1'

## NN model 불러와서 진단

텐서플로우 그래프 초기화 (아무것도 작성안했을 시 실행 X)

In [12]:
# Tensorflow Graph 지울 때 사용
tf.reset_default_graph() #구조를 미리 만들어 놓을걸 지움

In [13]:
# input place holders
X = tf.placeholder(tf.float32, [None, Training_Fold1.shape[1]])
Y = tf.placeholder(tf.float32, [None, 2])
#동일한 방식으로 구조 선언
# dropout (keep_prob) rate 0.7 on training, but should be 1 for testing
# keep_prob = tf.placeholder(tf.float32)

# weights & bias for nn layers

# Input Layer
W1 = tf.Variable(tf.random_normal([Training_Fold1.shape[1], noOfNeuron]))
b1 = tf.Variable(tf.random_normal([noOfNeuron]))
L1 = tf.matmul(X, W1) + b1

# Hidden Layer 1
W2 = tf.Variable(tf.random_normal([noOfNeuron, noOfNeuron]))
b2 = tf.Variable(tf.random_normal([noOfNeuron]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

# Hidden Layer 2
W3 = tf.Variable(tf.random_normal([noOfNeuron, noOfNeuron]))
b3 = tf.Variable(tf.random_normal([noOfNeuron]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

# Output Layer
W4 = tf.Variable(tf.random_normal([noOfNeuron, 2]))
b4 = tf.Variable(tf.random_normal([2]))
hypothesis = tf.matmul(L3, W4) + b4

# define cost/loss & optimizer
cost      = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learningRate).minimize(cost)

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy           = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [14]:
# 모델 불러오기 (그래프 파라미터 [Variable w, b] 불러옴)
sess  = tf.Session()
LoadedModel = tf.train.Saver()
LoadedModel.restore(sess, './MLmodels/ANN_model_tf1')

INFO:tensorflow:Restoring parameters from ./MLmodels/ANN_model_tf1


In [15]:
Test_Data  = np.array(pd.read_csv('./K_FoldData/Test_Data' , sep=",", header=None))
Test_Label = np.array(pd.read_csv('./K_FoldData/Test_Label_forANN', sep=",", header=None))

In [16]:
Accuracy = sess.run(accuracy, feed_dict={X: Test_Data, Y: Test_Label})
print('[Performance of ANN model] \n')
print('Accuracy : {:.2f}%'.format(Accuracy*100))

[Performance of ANN model] 

Accuracy : 97.50%
